In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
%matplotlib inline
from sklearn.model_selection import GridSearchCV
from matplotlib.pyplot import rcParams
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/tabular-playground-series-jul-2021'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/train.csv")

In [ ]:
test_df=pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/test.csv")

In [ ]:
train_df.columns

In [ ]:
test_df.columns

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.dropna().mean()

In [ ]:
num_col=train_df.select_dtypes(exclude="object").columns

In [ ]:
from scipy.stats import skew
train_df[num_col]=train_df[num_col].apply(lambda x:skew(x))

In [ ]:
a=(train_df.target_carbon_monoxide)
b=np.asanyarray(train_df.target_benzene)
c=np.asanyarray(train_df.target_nitrogen_oxides)

In [ ]:
a=np.log1p(a)
b=np.log1p(b)
c=np.log1p(c)

In [ ]:
train_df.shape

In [ ]:
train_df['absolute_humidity'].unique()

In [ ]:
x1=(train_df[['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4','sensor_5','absolute_humidity','relative_humidity','deg_C']])
x2=np.asanyarray(train_df[['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4','sensor_5','absolute_humidity','relative_humidity','deg_C']])
x3=np.asanyarray(train_df[['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4','sensor_5','absolute_humidity','relative_humidity','deg_C']])

In [ ]:
n_col=x1.corrwith(a).abs().sort_values(ascending=False).index
x1_num=x1[n_col]

In [ ]:
high_corr=x1_num.corrwith(a)[x1_num.corrwith(a).abs()>0.5]
x1_num=x1_num[high_corr]

In [ ]:
from sklearn import preprocessing
x1=preprocessing.StandardScaler().fit(x1).transform(x1)
x1[0:5]

In [ ]:
from sklearn import preprocessing
x2=preprocessing.StandardScaler().fit(x2).transform(x2)
x2[0:5]

In [ ]:
from sklearn import preprocessing
x3=preprocessing.StandardScaler().fit(x3).transform(x3)
x3[0:5]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x1,a,test_size=0.1,random_state=9)

In [ ]:
x_train1,x_test1,y_train1,y_test1=train_test_split(x2,b,test_size=0.1,random_state=9)

In [ ]:
x_train2,x_test2,y_train2,y_test2=train_test_split(x3,c,test_size=0.1,random_state=9)

In [ ]:
test_df.columns

In [ ]:
test_set=np.asanyarray(test_df[['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4','sensor_5','absolute_humidity','relative_humidity','deg_C']])
test_set[:5]

In [ ]:
from xgboost import XGBRegressor
p1={'n_estimators': 150,'objective':'reg:squarederror',
          'learning_rate': 0.1, 
          'max_depth': 4,'nthread':4,'random_state':42,'booster':'dart',
          'colsample_bytree': 0.75,'subsample': 0.75, 
          'colsample_bylevel': 0.75}
reg=XGBRegressor(**p1)
reg.fit(x1,a)
y_pred=reg.predict(test_set)

In [ ]:
from xgboost import XGBRegressor
p1={'n_estimators': 150,'objective':'reg:squarederror',
          'learning_rate': 0.1, 
          'max_depth': 4,'nthread':4,'random_state':42,'booster':'dart',
          'colsample_bytree': 0.75,'subsample': 0.75, 
          'colsample_bylevel': 0.75}
reg=XGBRegressor(**p1)
reg.fit(x2,b)
y_pred1=reg.predict(test_set)

In [ ]:
test_set1=np.asanyarray(test_df[['sensor_1', 'sensor_2', 'sensor_3', 'sensor_5']])

In [ ]:
from xgboost import XGBRegressor
p1={'n_estimators': 150,'objective':'reg:squarederror',
          'learning_rate': 0.1, 
          'max_depth': 4,'nthread':4,'random_state':42,'booster':'dart',
          'colsample_bytree': 0.75,'subsample': 0.75, 
          'colsample_bylevel': 0.75}
reg=XGBRegressor(**p1)
reg.fit(x3,c)
y_pred2=reg.predict(test_set)

In [ ]:
test_df['target_carbon_monoxide']=pd.Series(y_pred.reshape(1,-1)[0])
test_df['target_benzene']=pd.Series(y_pred1.reshape(1,-1)[0])
test_df['target_nitrogen_oxides']=pd.Series(y_pred2.reshape(1,-1)[0])
submission=pd.concat([test_df['date_time'],test_df['target_carbon_monoxide'],test_df['target_benzene'],test_df['target_nitrogen_oxides']],axis=1)
submission.to_csv('submission.csv',index=False)